### Employee Map Automization Tool

In [12]:
# 220713 Rendering Graduated Color
import arcpy, os, sys
arcpy.env.workspace = "C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb"
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps()[0]
aprxLyr = aprxMap.listLayers()[2]

sym = aprxLyr.symbology
sym.renderer.classificationField = "Join_Count"
sym.renderer.breakCount = 5
sym.renderer.colorRamp = aprx.listColorRamps('Cyan to Purple')[0]
aprxLyr.symbology = sym

# Save a Copy
aprx.saveACopy("GraduatedColors2.aprx")

In [5]:
import arcpy, os, sys
arcpy.env.workspace = "C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb"
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps()[0]
aprxMap.addDataFromPath(emp_zip_fly)

aprxLyr = aprxMap.listLayers()[0]
sym = aprxLyr.symbology



if hasattr(sym, 'renderer'):
    if sym.renderer.type == 'SimpleRenderer':
        sym.updateRenderer('GraduatedColorsRenderer')
        sym.renderer.classificationField = 'Join_Count'
        sym.renderer.breakCount = 5
        sym.renderer.colorRamp = aprx.listColorRamps('Yellow-Orange-Red (Continuous)')[0]
        
        aprxLyr.symbology = sym


In [6]:
# EMA Ver.4
# Updates Note
# 220713 
# 1. Creating Mean Center
#   1) Create meidan center
#   2) Capture the poitns within the radius to exclude outliers
#   3) Create mean center point with the selected pop-centered employee points
# 2. Rendering the employee count layer 


import arcpy
from arcpy import env
import os
import sys

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Add the Zipcode Feature layer to Map
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps()[0]
aprxMap.addDataFromPath(emp_zip_fly)

# Rendering the Zipcode Feature layer
aprxLyr = aprxMap.listLayers()[0]
sym = aprxLyr.symbology
if hasattr(sym, 'renderer'):
    if sym.renderer.type == 'SimpleRenderer':
        sym.updateRenderer('GraduatedColorsRenderer')
        sym.renderer.classificationField = 'Join_Count'
        sym.renderer.breakCount = 5
        sym.renderer.colorRamp = aprx.listColorRamps('Yellow-Orange-Red (Continuous)')[0]
        
        aprxLyr.symbology = sym

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Input Rows: Value is required
WARNING 000725: Output Table: Dataset C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\emp_table already exists.
Failed to execute (CopyRows).


In [ ]:
# EMA Ver.4
# Updates Note
# 220713 
# 1. Creating Mean Center
#   1) Create meidan center
#   2) Capture the poitns within the radius to exclude outliers
#   3) Create mean center point with the selected pop-centered employee points
# 2. Rendering the employee count layer 


import arcpy
from arcpy import env
import os

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)

##### July 13th, 2022 - EMA Ver.4

In [ ]:
# EMA Ver.4
# Updates Note
# 220713 
# 1. Creating Mean Center
#   1) Create meidan center
#   2) Capture the poitns within the radius to exclude outliers
#   3) Create mean center point with the selected pop-centered employee points
# 2. Rendering the employee count layer 


import arcpy
from arcpy import env
import os

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)

##### July 6th, 2022 - EMA Ver.3.1

In [ ]:
# The Ver 3.1 for comparative analysis will calculate the mean center 
#  using Mean Center

# import system modules
import arcpy
from arcpy import env
import os

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
emp_count_xlsx = arcpy.GetParameterAsText(6) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
arcpy.MeanCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_mean_fly,
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)

##### June 27th, 2022 - EMA Ver.3

In [ ]:
# import system modules
import arcpy
from arcpy import env
import os

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_medcent_fly = arcpy.GetParameterAsText(5) # feature layer
emp_count_xlsx = arcpy.GetParameterAsText(6) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_medcent_fly,
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)

##### June 8th, 2022 - EMA Ver.2

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_count_xlsx = arcpy.GetParameterAsText(5) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = "join_table"
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)

##### May 24th, 2022

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
#arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/base.gdb"
#workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_count_xlsx = arcpy.GetParameterAsText(5) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = "join_table"
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)

##### May 19th, 2022

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"
workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
emp_csv = arcpy.GetParameterAsText(0) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(1) # employee csv field
pop_csv = arcpy.GetParameterAsText(2) # population center zipcode csv, Taalbe or Table View
pop_field = arcpy.GetParameterAsText(3)
xy_point_fly = arcpy.GetParameterAsText(4) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(5) # feature layer

# Copy rows to create OID in memory
# emp_table = "emp_table"
emp_table = os.path.join(workspace, "emp_table")
pop_table = "pop_table"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, pop_field)

# Copy the outcome
join_table = "join_table"
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join
zip_polyg_fly = "C:/ArcGIS/Business Analyst/US_2021/Data/Demographic Data/USA_ACS_2021.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete("point.shp")

##### May 18th, 2022

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
emp_csv = arcpy.GetParameterAsText(0) # employee csv, Table view
pop_csv = arcpy.GetParameterAsText(1) # population center csv, Table View
emp_table = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/emp_table"
pop_table = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/pop_table"
join_field = "ZipCode"
join_table = arcpy.GetParameterAsText(2) # joined and copied table, table
xy_point = arcpy.GetParameterAsText(3) # feature class, shapefile

# Copy rows to create OID in memory
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
arcpy.JoinField_management(emp_table, join_field, 
                           pop_table, join_field)

# Copy the outcome
arcpy.Copy_management(emp_table, join_table)

# Create point features
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))


##### May 17h, 2022

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
emp_csv = r"C:\ArcGIS\Projects\405555\COPY\Dev_405555_copied_csv.csv"
pop_csv = r"C:\ArcGIS\Projects\ZipCode\PopCenterZips_March2022_csv.csv"
join_field = "ZipCode"
xy_point = arcpy.GetParameterAsText(0) # feature class, shapefile
output_table = arcpy.GetParameterAsText(1)

# Join the two table
arcpy.JoinField_management(emp_csv, join_field, 
                           pop_csv, join_field)

# Copy the outcome
arcpy.Copy_management(emp_table, join_table)

# Create point features
arcpy.management.XYTableToPoint(
    in_table = emp_csv, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Delte 

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
xy_excel = arcpy.GetParameterAsText(0) # input excel, File (Filter - file, 'xlsx' extension)
xy_point = arcpy.GetParameterAsText(1) # feature class, shapefile

# Create point features
arcpy.management.XYTableToPoint(
    in_table = xy_excel, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
emp_excel = arcpy.GetParameterAsText(0) # input excel, File (Filter - file, 'xlsx' extension)
emp_table = arcpy.GetParameterAsText(1) # employee table, Table or Table View
emp_field = arcpy.GetParameterAsText(2) # employee field, field (dependent on emp_table)
pop_excel = arcpy.GetParameterAsText(3) # popCenter excel, File (Filter - file, 'xlsx ' extension)
pop_table = arcpy.GetParameterAsText(4) # popCenter table, Table or Table View
pop_field = arcpy.GetParameterAsText(5) # popCenter field, field (dependent on pop_table)
join_table = arcpy.GetParameterAsText(6) # joined and copied table, table
join_point = arcpy.GetParameterAsText(7) # joined points shp, feature class

# Create tables from csv files with ExcelToTable
arcpy.ExcelToTable_conversion(emp_excel, emp_table, "Sheet1")
arcpy.ExcelToTable_conversion(pop_excel, pop_table, "Sheet1")

# Join the two table
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, pop_field)

# Copy the outcome
arcpy.Copy_management(emp_table, join_table)

# Create point features
arcpy.management.XYTableToPoint(join_table, join_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))

#### Goal 1-2. Create a point feature layer from a joined table

In [ ]:
# reference for creating xy from table


# 1. XYTableToPoint
# create a point from table with spatial reference
arcpy.management.XYTableToPoint(inJoin_table, out_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))


# 2. MakeXYEventLayer
# import system modules 
import arcpy

# Set environment settings
arcpy.env.workspace = "C:/data"
 
# Set the local variables
in_Table = "firestations.dbf"
x_coords = "POINT_X"
y_coords = "POINT_Y"
z_coords = "POINT_Z"
out_Layer = "firestations_layer"
saved_Layer = r"c:\output\firestations.lyr"

# Set the spatial reference
spRef = r"Coordinate Systems\Projected Coordinate Systems\Utm\Nad 1983\NAD 1983 UTM Zone 11N.prj"

# Make the XY event layer...
arcpy.MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, 
                                  spRef, z_coords)

# Save to a layer file
arcpy.SaveToLayerFile_management(out_Layer, saved_Layer)


##### May 15th, 2022

#### Goal 1-1. Create a joined table from csv files

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
emp_excel = arcpy.GetParameterAsText(0) # input excel, File (Filter - file, 'xlsx' extension)
emp_table = arcpy.GetParameterAsText(1) # employee table, Table or Table View
emp_field = arcpy.GetParameterAsText(2) # employee field, field (dependent on emp_table)
pop_excel = arcpy.GetParameterAsText(3) # popCenter excel, File (Filter - file, 'xlsx ' extension)
pop_table = arcpy.GetParameterAsText(4) # popCenter table, Table or Table View
pop_field = arcpy.GetParameterAsText(5) # popCenter field, field (dependent on pop_table)
join_table = arcpy.GetParameterAsText(6) # joined and copied table, table
join_point = arcpy.GetParameterAsText(7) # joined points shp, feature class

# Create tables from csv files with ExcelToTable
arcpy.ExcelToTable_conversion(emp_excel, emp_table, "Sheet1")
arcpy.ExcelToTable_conversion(pop_excel, pop_table, "Sheet1")

# Join the two table
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, pop_field)

# Copy the outcome
arcpy.Copy_management(emp_table, join_table)

# Create point features
arcpy.management.XYTableToPoint(join_table, join_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))


In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
in_excel = arcpy.GetParameterAsText(0) # input excel, File (Filter - file, 'xlsx' extension)
out_table = arcpy.GetParameterAsText(1) # output table, Table
#out_field = arcpy.GetParameterAsText(2) # output field, field
join_table = arcpy.GetParameterAsText(2) # output joined table, Table

# Create a table from csv file with ExcelToTable
arcpy.ExcelToTable_conversion(in_excel, out_table, "Sheet1")

# Create table views from the two table to join them
arcpy.MakeTableView_management(out_table, "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/employee_tview")
arcpy.MakeTableView_management("C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/popCenter_table", "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/popCenter_tview")
# Completes by this step ------------------------------------

# Join the two table
# Condition 1. Create table view files and try table join
# Condition 2. If it successes, try the emporary file with in-memory
arcpy.AddJoin_management("C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/employee_tview", "ZipCode",
                         "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/popCenter_tview", "ZipCode")

# Copy the outcome
arcpy.Copy_management("C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb/employee_tview", join_table)


In [ ]:
# t1-3. change fields froms text to field
# t2-0. if t1-3 fails, search about the csv and file values

# import system modules
import arcpy
import os
import xlrd
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
in_excel = arcpy.GetParameterAsText(0) # input excel, File
in_sheet = arcpy.GetParameterAsText(1) # input sheet, String
out_table = arcpy.GetParameterAsText(2) # output table, Table

# in_field = arcpy.GetParameterAsText(1) # input table field
# join_table = arcpy.GetParameterAsText(2) # join table, csv - any values
# join_field = arcpy.GetParameterAsText(3) # join table field
# out_point = arcpy.GetParameterAsText(4) # feature class

# Create a table from csv file with ExcelToTable
arcpy.ExcelToTable_conversion(in_excel, out_table, in_sheet)

# createtemporary copied table views
#arcpy.MakeTableView_management(in_table, r"memory\employees_tview")
#arcpy.MakeTableView_management(join_table, r"memory\popCenter_tview")

# table join with PopCenterZip
#inJoin_table = arcpy.AddJoin_management(r"memory\employees_tview", in_field, 
#                                        r"memory\popCenter_tview", join_field)
# delete the temporary parameters in memory
#arcpy.Delete_management(r"memory\employees_tview")
#arcpy.Delete_management(r"memory\popCenter_tview") 

In [ ]:
# create in memory path
memory = 'in_memory'
mem_table_path = os.path.join(memory, 'csv')

#create the in memory table
arcpy.CopyRows_management(PathToYourCSV, mem_table_path)   

#do the join
arcpy.JoinField_management(in_fc_path, in_field, mem_table_path, join_field, [fields])

# arcpy delete
arcpy.Delete_management(out_data)


##### May 13th, 2022

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True
arcpy.env.qualifiedFieldNames = False 
   # Output field name includes the table name

# set parameters
in_table = arcpy.GetParameterAsText(0) # input table view
in_field = arcpy.GetParameterAsText(1) # input table field
join_table = arcpy.GetParameterAsText(2) # join table
join_field = arcpy.GetParameterAsText(3) # join table field
out_point = arcpy.GetParameterAsText(4) # feature class

# table join with PopCenterZip
in_join_table = arcpy.AddJoin_management(in_table, in_field, 
                                        join_table, join_field)

# create a point from table with spatial reference
arcpy.management.XYTableToPoint(in_join_table, out_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))


In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True
arcpy.env.qualifiedFieldNames = False 
   # Output field name includes the table name

# set parameters
in_table = arcpy.GetParameterAsText(0) # table view
in_field = arcpy.GetParameterAsText(1) # field
join_table = "C:\ArcGIS\Projects\Dev_Proto\Dev_Proto.gdb\PopCenterZips_Table" # join table
join_field = "ZipCode"
out_point = arcpy.GetParameterAsText(2) # feature class

# table join with PopCenterZip
in_join_table = arcpy.AddJoin_management(in_table, in_field, 
                                        join_table, join_field)

# create a point from table with spatial reference
arcpy.management.XYTableToPoint(in_join_table, out_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))
